In [2]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm

In [18]:
def api_call(ticker):
    url = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=annual&apikey=FCazkcdySuLOf9TJiEMOaqykEB4Du8f1'
    response = requests.get(url)
    data = response.json()
    pd.set_option('display.max_columns', None)
    df = pd.DataFrame(data) 
    return df

ticker = 'ABC'
df = api_call(ticker)

In [4]:
def required_informations(df):
    firm_value = df['totalAssets'][0]
    risk_free_rate = 0.03
    time = 1
    df['assets_log_return'] = np.log(df['totalAssets']/df['totalAssets'].shift(1))
    vol= df['assets_log_return'].std()
    return firm_value, risk_free_rate, vol, time

A, mu, sigma, t = required_informations(df)

In [5]:
def ead(rating,bond,commitment):              
    rating_dict = {                            #the UGD lookup is from Internal credit risk model
        "AAA":0.69,"AA":0.73,"A":0.71,
        "BBB":0.65,"BB":0.52,"B":0.48,
        "CCC":0.44
    }
    EAD = bond + rating_dict[rating] * commitment
    return EAD

EAD = ead("BBB",500000000,500000000)
print(f'The exposure at default:- {EAD}')

The exposure at default:- 825000000.0


In [6]:
def default_point_calculations(df):
    long_current_liabilities_max = df['totalNonCurrentLiabilities'].max()
    short_liabilities_max = df['totalCurrentLiabilities'].max()
    default_point = short_liabilities_max  + long_current_liabilities_max * 0.5
    return default_point 

default_point = default_point_calculations(df)

In [7]:
def calculate_PD(A,L,sigma,mu,t):
    numerator = np.log(A/L) + (mu - 0.5*sigma**2)*(t)
    denominator = sigma * np.sqrt(t);
    DD = numerator/denominator
    np_1 = norm.cdf(DD)
    PD = 1 - np_1
    return PD * 100

PD = calculate_PD(A, default_point, sigma, mu, t)
print(f'The probability Default is:- {PD} %')

The probability Default is:- 13.850479647036506 %


In [8]:
avg_LGD_on_bonds = 0.5637
std = 0.0104
max_for_bonds = 1.1
def LGD_calculations(avg, std, max_value):
    alpha = (avg/max_value)*((avg*(max_value-avg)/max_value*std**2)-1)
    beta  = alpha*((max_value/avg)-1)
    mean_recovery = alpha/(alpha + beta)
    LGD = 1 - mean_recovery
    return LGD

LGD = LGD_calculations(avg_LGD_on_bonds, std, max_for_bonds)*100
print(f'The Loss Giving Default is:- {LGD} %')

The Loss Giving Default is:- 48.754545454545465 %


In [9]:
def exposure_at_default(EAD, PD, LGD):
    EL = (EAD * LGD * PD)/1e4
    return EL
EL = exposure_at_default(EAD, PD, LGD)
print(f'The Expected loss is:- ${EL} ')

The Expected loss is:- $55710091.760292605 
